**Criando conexão no sistema.**

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session
from pyspark.sql import functions as F

# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()


/usr/local/lib/python3.12/dist-packages/google/cloud/dataproc_spark_connect/session.py:332: UserWarning: Python version mismatch detected: Client is using Python 3.12, but Dataproc runtime 3.0 uses Python 3.11. This mismatch may cause issues with Python UDF (User Defined Function) compatibility. Consider using Python 3.11 for optimal UDF execution.
  dataproc_config: Session = self._get_dataproc_config()


██████████████████████████████████████████████████▏                             

In [ ]:
data_inicio = '2023-01-01'
data_fim = '2026-12-31'

In [ ]:
df_datas = spark.sql(f"select explode(sequence(to_date('{data_inicio}'), to_date('{data_fim}'), interval 1 day)) as full_date")


**Criando uma dimensão de datas**

In [ ]:
df_dim_date = (
    df_datas
    .withColumn("date_key", F.date_format("full_date", "yyyyMMdd").cast("int"))
    .withColumn("year", F.year("full_date"))
    .withColumn("month", F.month("full_date"))
    .withColumn("month_name", F.date_format("full_date", "MMMM"))
    .withColumn("day", F.dayofmonth("full_date"))
    .withColumn("day_of_week", F.dayofweek("full_date"))
    .withColumn("day_of_week_name", F.date_format("full_date", "EEEE"))
    .withColumn("quarter", F.quarter("full_date"))
    .withColumn("week_of_year", F.weekofyear("full_date"))
    .withColumn("day_of_year", F.date_format("full_date", "D"))
    .withColumn("is_weekday", (F.dayofweek("full_date") >= 2) & (F.dayofweek("full_date") <= 6))
)


In [ ]:
df_dim_date = (
    df_dim_data
    .withColumn("year_month", F.date_format("full_date", "yyyy-MM"))
    .withColumn("year_quarter", F.date_format("full_date", "yyyy-QQ"))
)


**Salvando dataframe na Gold.**

In [ ]:
(df_dim_date.write
        .format("bigquery")
        .option("table", "clinic-de.3_gold.dim_date")
        .option("writeMethod", "direct")
        .mode("overwrite")
        .save()
    )

print("----------------------Tabela salva com sucesso----------------------")

----------------------Tabela salva com sucesso----------------------


In [ ]:
spark.stop()